In [1]:
# Saving the Best model...

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SequentialFeatureSelector

#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.feature_selection import RFE


In [2]:
def backward_selection(indep, dep,n):
    logistic_model=LogisticRegression(solver='saga', max_iter=10)
    
    fs = SequentialFeatureSelector(estimator=logistic_model, n_features_to_select=n, direction='backward', scoring=None, cv=5, n_jobs=1)
    
    fit1 = fs.fit(indep, dep)
    x_new= fit1.transform(indep)
    
    selected_features= indep.columns[fit1.get_support()]
    
    return x_new, selected_features


def Split_Scaler(indep,dep):
    x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
    scX=StandardScaler()
    x_train=scX.fit_transform(x_train)
    x_test=scX.transform(x_test)
    return x_train,x_test,y_train,y_test,scX

#    # save Standard scalar obj
#    pickle.dump(scX,open("scx.pkl",'wb'))

def cm_predition(classifier, x_test):
    y_pred=classifier.predict(x_test)

    #making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)

    # making Classification Report
    from sklearn.metrics import classification_report
    clf_report=classification_report(y_test,y_pred)

    # finding the accuracy score
    from sklearn.metrics import accuracy_score
    accuracy_score=accuracy_score(y_test,y_pred)
    
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    

# Displaying the values in the tabular frame

def BW_selection(accuracy_score):
    
    df=pd.DataFrame(index=["Logistic"],columns=["Logistic"])
    df["Logistic"]=accuracy_score
    
    return df




In [3]:
dataset=pd.read_csv("prep.csv",index_col=None)
df1=dataset  

#Convert categorical variable into dummy/indicator variables.
df1=pd.get_dummies(df1,drop_first=True,dtype=int)
df1

# separation of independent and dependent variables
indep=df1.drop("classification_yes",axis=1)
dep=df1["classification_yes"]
  
    
x_new, selected_features =backward_selection (indep, dep,4)



/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/py

In [4]:
x_train,x_test,y_train,y_test,scX = Split_Scaler(x_new, dep) 

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = logistic(x_train,y_train,x_test)

result= BW_selection(accuracy_score)

In [5]:
print("Accuracy: ",accuracy_score)

Accuracy:  0.9416666666666667


In [6]:
result #4

,Logistic
Logistic,0.941667


In [7]:
print ("seleceted features: ",selected_features)

seleceted features:  Index(['bgr', 'bu', 'hrmo', 'pcv'], dtype='object')


In [8]:
# Save the model

pickle.dump(classifier,open("log_model02","wb"))

# save Standard scalar obj
pickle.dump(scX,open("scx02.pkl",'wb'))

In [9]:
df1[selected_features]

,bgr,bu,hrmo,pcv
0,148.112676,57.482105,12.518156,38.868902
1,148.112676,22.000000,10.700000,34.000000
2,99.000000,23.000000,12.000000,34.000000
3,148.112676,16.000000,8.100000,38.868902
4,148.112676,25.000000,11.800000,36.000000
...,...,...,...,...
394,219.000000,36.000000,12.500000,37.000000
395,220.000000,68.000000,8.700000,27.000000
396,110.000000,115.000000,9.100000,26.000000
397,207.000000,80.000000,8.500000,38.868902
